In [1]:
import pandas as pd
import mysql.connector
import numpy as np

# --- Configuration (replace with your actual details) ---
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '**********',
    'database': 'vaccine'
}

# Excel file path and the target MySQL table name
EXCEL_FILE =r"E:\project\project\csv_file\vaccine\clean\vaccine-schedule-data.xlsx"
TABLE_NAME = 'vac_sch_data'

def get_sql_type(column_data):
    """Dynamically determines the best SQL data type for a pandas column."""
    dtype = column_data.dtype

    if pd.api.types.is_integer_dtype(dtype):
        # Use BIGINT for all integer types to be safe.
        return 'BIGINT'
    if pd.api.types.is_float_dtype(dtype):
        # Use DOUBLE for float types.
        return 'DOUBLE'
    if pd.api.types.is_datetime64_any_dtype(dtype):
        # Use DATETIME for dates and times.
        return 'DATETIME'
    if pd.api.types.is_string_dtype(dtype):
        # For string/object columns, find the max length.
        max_len = int(column_data.str.len().max())
        
        # If the column is empty, default to VARCHAR(255).
        if pd.isna(max_len):
            return 'VARCHAR(255)'
            
        # If max length is very large, use TEXT. 16383 is a safe threshold.
        if max_len > 16383:
            return 'TEXT'
            
        # Otherwise, create a VARCHAR with a size slightly larger than the max length found.
        # Default to at least 255.
        varchar_size = max(255, max_len + 10)
        return f'VARCHAR({varchar_size})'
        
    # Fallback for any other data types.
    return 'TEXT'

def import_excel_to_mysql(excel_file, table_name):
    """
    Reads data from an Excel file and imports it into a MySQL table,
    dynamically creating column types based on the data.
    """
    conn = None
    cursor = None
    try:
        df = pd.read_excel(excel_file)
        df = df.where(pd.notna(df), None)

        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()

        cursor.execute(f"DROP TABLE IF EXISTS `{table_name}`")
        
        # Dynamically build the CREATE TABLE statement with smart type detection.
        column_definitions = []
        for col in df.columns:
            col_name_sanitized = f"`{col.replace('`', '``')}`"
            sql_type = get_sql_type(df[col])
            column_definitions.append(f"{col_name_sanitized} {sql_type}")
        
        create_table_sql = f"CREATE TABLE `{table_name}` ({', '.join(column_definitions)})"
        cursor.execute(create_table_sql)
        print(f"Table '{table_name}' created successfully with dynamic column types.")

        # Prepare and execute the bulk insert.
        data_to_insert = [tuple(row) for row in df.itertuples(index=False)]
        placeholders = ', '.join(['%s'] * len(df.columns))
        column_names = ', '.join([f"`{col.replace('`', '``')}`" for col in df.columns])
        insert_sql = f"INSERT INTO `{table_name}` ({column_names}) VALUES ({placeholders})"
        
        cursor.executemany(insert_sql, data_to_insert)
        conn.commit()
        
        print(f"{cursor.rowcount} records were imported successfully into '{table_name}'!")

    except FileNotFoundError:
        print(f"Error: The file at '{excel_file}' was not found.")
    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        if conn and conn.is_connected():
            conn.rollback()
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        if conn and conn.is_connected():
            if cursor:
                cursor.close()
            conn.close()
            print("MySQL connection has been closed.")

if __name__ == "__main__":
    import_excel_to_mysql(EXCEL_FILE, TABLE_NAME)


Table 'vac_sch_data' created successfully with dynamic column types.
8052 records were imported successfully into 'vac_sch_data'!
MySQL connection has been closed.


In [3]:
import pandas as pd
import mysql.connector
import numpy as np

# --- Configuration (replace with your actual details) ---
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '**********',
    'database': 'vaccine'
}

# Excel file path and the target MySQL table name
EXCEL_FILE =r"E:\project\project\csv_file\vaccine\clean\reported-cases-data.xlsx"
TABLE_NAME = 'rc_data'

def get_sql_type(column_data):
    """Dynamically determines the best SQL data type for a pandas column."""
    dtype = column_data.dtype

    if pd.api.types.is_integer_dtype(dtype):
        # Use BIGINT for all integer types to be safe.
        return 'BIGINT'
    if pd.api.types.is_float_dtype(dtype):
        # Use DOUBLE for float types.
        return 'DOUBLE'
    if pd.api.types.is_datetime64_any_dtype(dtype):
        # Use DATETIME for dates and times.
        return 'DATETIME'
    if pd.api.types.is_string_dtype(dtype):
        # For string/object columns, find the max length.
        max_len = int(column_data.str.len().max())
        
        # If the column is empty, default to VARCHAR(255).
        if pd.isna(max_len):
            return 'VARCHAR(255)'
            
        # If max length is very large, use TEXT. 16383 is a safe threshold.
        if max_len > 16383:
            return 'TEXT'
            
        # Otherwise, create a VARCHAR with a size slightly larger than the max length found.
        # Default to at least 255.
        varchar_size = max(255, max_len + 10)
        return f'VARCHAR({varchar_size})'
        
    # Fallback for any other data types.
    return 'TEXT'

def import_excel_to_mysql(excel_file, table_name):
    """
    Reads data from an Excel file and imports it into a MySQL table,
    dynamically creating column types based on the data.
    """
    conn = None
    cursor = None
    try:
        df = pd.read_excel(excel_file)
        df = df.where(pd.notna(df), None)

        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()

        cursor.execute(f"DROP TABLE IF EXISTS `{table_name}`")
        
        # Dynamically build the CREATE TABLE statement with smart type detection.
        column_definitions = []
        for col in df.columns:
            col_name_sanitized = f"`{col.replace('`', '``')}`"
            sql_type = get_sql_type(df[col])
            column_definitions.append(f"{col_name_sanitized} {sql_type}")
        
        create_table_sql = f"CREATE TABLE `{table_name}` ({', '.join(column_definitions)})"
        cursor.execute(create_table_sql)
        print(f"Table '{table_name}' created successfully with dynamic column types.")

        # Prepare and execute the bulk insert.
        data_to_insert = [tuple(row) for row in df.itertuples(index=False)]
        placeholders = ', '.join(['%s'] * len(df.columns))
        column_names = ', '.join([f"`{col.replace('`', '``')}`" for col in df.columns])
        insert_sql = f"INSERT INTO `{table_name}` ({column_names}) VALUES ({placeholders})"
        
        cursor.executemany(insert_sql, data_to_insert)
        conn.commit()
        
        print(f"{cursor.rowcount} records were imported successfully into '{table_name}'!")

    except FileNotFoundError:
        print(f"Error: The file at '{excel_file}' was not found.")
    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        if conn and conn.is_connected():
            conn.rollback()
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        if conn and conn.is_connected():
            if cursor:
                cursor.close()
            conn.close()
            print("MySQL connection has been closed.")

if __name__ == "__main__":
    import_excel_to_mysql(EXCEL_FILE, TABLE_NAME)


Table 'rc_data' created successfully with dynamic column types.
84869 records were imported successfully into 'rc_data'!
MySQL connection has been closed.


In [5]:
import pandas as pd
import mysql.connector
import numpy as np

# --- Configuration (replace with your actual details) ---
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '**********',
    'database': 'vaccine'
}

# Excel file path and the target MySQL table name
EXCEL_FILE =r"E:\project\project\csv_file\vaccine\clean\incidence-rate-data.xlsx"
TABLE_NAME = 'ir_data'

def get_sql_type(column_data):
    """Dynamically determines the best SQL data type for a pandas column."""
    dtype = column_data.dtype

    if pd.api.types.is_integer_dtype(dtype):
        # Use BIGINT for all integer types to be safe.
        return 'BIGINT'
    if pd.api.types.is_float_dtype(dtype):
        # Use DOUBLE for float types.
        return 'DOUBLE'
    if pd.api.types.is_datetime64_any_dtype(dtype):
        # Use DATETIME for dates and times.
        return 'DATETIME'
    if pd.api.types.is_string_dtype(dtype):
        # For string/object columns, find the max length.
        max_len = int(column_data.str.len().max())
        
        # If the column is empty, default to VARCHAR(255).
        if pd.isna(max_len):
            return 'VARCHAR(255)'
            
        # If max length is very large, use TEXT. 16383 is a safe threshold.
        if max_len > 16383:
            return 'TEXT'
            
        # Otherwise, create a VARCHAR with a size slightly larger than the max length found.
        # Default to at least 255.
        varchar_size = max(255, max_len + 10)
        return f'VARCHAR({varchar_size})'
        
    # Fallback for any other data types.
    return 'TEXT'

def import_excel_to_mysql(excel_file, table_name):
    """
    Reads data from an Excel file and imports it into a MySQL table,
    dynamically creating column types based on the data.
    """
    conn = None
    cursor = None
    try:
        df = pd.read_excel(excel_file)
        df = df.where(pd.notna(df), None)

        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()

        cursor.execute(f"DROP TABLE IF EXISTS `{table_name}`")
        
        # Dynamically build the CREATE TABLE statement with smart type detection.
        column_definitions = []
        for col in df.columns:
            col_name_sanitized = f"`{col.replace('`', '``')}`"
            sql_type = get_sql_type(df[col])
            column_definitions.append(f"{col_name_sanitized} {sql_type}")
        
        create_table_sql = f"CREATE TABLE `{table_name}` ({', '.join(column_definitions)})"
        cursor.execute(create_table_sql)
        print(f"Table '{table_name}' created successfully with dynamic column types.")

        # Prepare and execute the bulk insert.
        data_to_insert = [tuple(row) for row in df.itertuples(index=False)]
        placeholders = ', '.join(['%s'] * len(df.columns))
        column_names = ', '.join([f"`{col.replace('`', '``')}`" for col in df.columns])
        insert_sql = f"INSERT INTO `{table_name}` ({column_names}) VALUES ({placeholders})"
        
        cursor.executemany(insert_sql, data_to_insert)
        conn.commit()
        
        print(f"{cursor.rowcount} records were imported successfully into '{table_name}'!")

    except FileNotFoundError:
        print(f"Error: The file at '{excel_file}' was not found.")
    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        if conn and conn.is_connected():
            conn.rollback()
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        if conn and conn.is_connected():
            if cursor:
                cursor.close()
            conn.close()
            print("MySQL connection has been closed.")

if __name__ == "__main__":
    import_excel_to_mysql(EXCEL_FILE, TABLE_NAME)


Table 'ir_data' created successfully with dynamic column types.
84945 records were imported successfully into 'ir_data'!
MySQL connection has been closed.


In [7]:
import pandas as pd
import mysql.connector
import numpy as np

# --- Configuration (replace with your actual details) ---
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '**********',
    'database': 'vaccine'
}

# Excel file path and the target MySQL table name
EXCEL_FILE =r"E:\project\project\csv_file\vaccine\clean\vaccine-introduction-data.xlsx"
TABLE_NAME = 'vi_data'

def get_sql_type(column_data):
    """Dynamically determines the best SQL data type for a pandas column."""
    dtype = column_data.dtype

    if pd.api.types.is_integer_dtype(dtype):
        # Use BIGINT for all integer types to be safe.
        return 'BIGINT'
    if pd.api.types.is_float_dtype(dtype):
        # Use DOUBLE for float types.
        return 'DOUBLE'
    if pd.api.types.is_datetime64_any_dtype(dtype):
        # Use DATETIME for dates and times.
        return 'DATETIME'
    if pd.api.types.is_string_dtype(dtype):
        # For string/object columns, find the max length.
        max_len = int(column_data.str.len().max())
        
        # If the column is empty, default to VARCHAR(255).
        if pd.isna(max_len):
            return 'VARCHAR(255)'
            
        # If max length is very large, use TEXT. 16383 is a safe threshold.
        if max_len > 16383:
            return 'TEXT'
            
        # Otherwise, create a VARCHAR with a size slightly larger than the max length found.
        # Default to at least 255.
        varchar_size = max(255, max_len + 10)
        return f'VARCHAR({varchar_size})'
        
    # Fallback for any other data types.
    return 'TEXT'

def import_excel_to_mysql(excel_file, table_name):
    """
    Reads data from an Excel file and imports it into a MySQL table,
    dynamically creating column types based on the data.
    """
    conn = None
    cursor = None
    try:
        df = pd.read_excel(excel_file)
        df = df.where(pd.notna(df), None)

        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()

        cursor.execute(f"DROP TABLE IF EXISTS `{table_name}`")
        
        # Dynamically build the CREATE TABLE statement with smart type detection.
        column_definitions = []
        for col in df.columns:
            col_name_sanitized = f"`{col.replace('`', '``')}`"
            sql_type = get_sql_type(df[col])
            column_definitions.append(f"{col_name_sanitized} {sql_type}")
        
        create_table_sql = f"CREATE TABLE `{table_name}` ({', '.join(column_definitions)})"
        cursor.execute(create_table_sql)
        print(f"Table '{table_name}' created successfully with dynamic column types.")

        # Prepare and execute the bulk insert.
        data_to_insert = [tuple(row) for row in df.itertuples(index=False)]
        placeholders = ', '.join(['%s'] * len(df.columns))
        column_names = ', '.join([f"`{col.replace('`', '``')}`" for col in df.columns])
        insert_sql = f"INSERT INTO `{table_name}` ({column_names}) VALUES ({placeholders})"
        
        cursor.executemany(insert_sql, data_to_insert)
        conn.commit()
        
        print(f"{cursor.rowcount} records were imported successfully into '{table_name}'!")

    except FileNotFoundError:
        print(f"Error: The file at '{excel_file}' was not found.")
    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        if conn and conn.is_connected():
            conn.rollback()
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        if conn and conn.is_connected():
            if cursor:
                cursor.close()
            conn.close()
            print("MySQL connection has been closed.")

if __name__ == "__main__":
    import_excel_to_mysql(EXCEL_FILE, TABLE_NAME)


Table 'vi_data' created successfully with dynamic column types.
138320 records were imported successfully into 'vi_data'!
MySQL connection has been closed.


In [9]:
import pandas as pd
import mysql.connector
import numpy as np

# --- Configuration (replace with your actual details) ---
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '**********',
    'database': 'vaccine'
}

# Excel file path and the target MySQL table name
EXCEL_FILE =r"E:\project\project\csv_file\vaccine\clean\coverage-data.xlsx"
TABLE_NAME = 'cov_data'

def get_sql_type(column_data):
    """Dynamically determines the best SQL data type for a pandas column."""
    dtype = column_data.dtype

    if pd.api.types.is_integer_dtype(dtype):
        # Use BIGINT for all integer types to be safe.
        return 'BIGINT'
    if pd.api.types.is_float_dtype(dtype):
        # Use DOUBLE for float types.
        return 'DOUBLE'
    if pd.api.types.is_datetime64_any_dtype(dtype):
        # Use DATETIME for dates and times.
        return 'DATETIME'
    if pd.api.types.is_string_dtype(dtype):
        # For string/object columns, find the max length.
        max_len = int(column_data.str.len().max())
        
        # If the column is empty, default to VARCHAR(255).
        if pd.isna(max_len):
            return 'VARCHAR(255)'
            
        # If max length is very large, use TEXT. 16383 is a safe threshold.
        if max_len > 16383:
            return 'TEXT'
            
        # Otherwise, create a VARCHAR with a size slightly larger than the max length found.
        # Default to at least 255.
        varchar_size = max(255, max_len + 10)
        return f'VARCHAR({varchar_size})'
        
    # Fallback for any other data types.
    return 'TEXT'

def import_excel_to_mysql(excel_file, table_name):
    """
    Reads data from an Excel file and imports it into a MySQL table,
    dynamically creating column types based on the data.
    """
    conn = None
    cursor = None
    try:
        df = pd.read_excel(excel_file)
        df = df.where(pd.notna(df), None)

        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()

        cursor.execute(f"DROP TABLE IF EXISTS `{table_name}`")
        
        # Dynamically build the CREATE TABLE statement with smart type detection.
        column_definitions = []
        for col in df.columns:
            col_name_sanitized = f"`{col.replace('`', '``')}`"
            sql_type = get_sql_type(df[col])
            column_definitions.append(f"{col_name_sanitized} {sql_type}")
        
        create_table_sql = f"CREATE TABLE `{table_name}` ({', '.join(column_definitions)})"
        cursor.execute(create_table_sql)
        print(f"Table '{table_name}' created successfully with dynamic column types.")

        # Prepare and execute the bulk insert.
        data_to_insert = [tuple(row) for row in df.itertuples(index=False)]
        placeholders = ', '.join(['%s'] * len(df.columns))
        column_names = ', '.join([f"`{col.replace('`', '``')}`" for col in df.columns])
        insert_sql = f"INSERT INTO `{table_name}` ({column_names}) VALUES ({placeholders})"
        
        cursor.executemany(insert_sql, data_to_insert)
        conn.commit()
        
        print(f"{cursor.rowcount} records were imported successfully into '{table_name}'!")

    except FileNotFoundError:
        print(f"Error: The file at '{excel_file}' was not found.")
    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        if conn and conn.is_connected():
            conn.rollback()
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        if conn and conn.is_connected():
            if cursor:
                cursor.close()
            conn.close()
            print("MySQL connection has been closed.")

if __name__ == "__main__":
    import_excel_to_mysql(EXCEL_FILE, TABLE_NAME)


Table 'cov_data' created successfully with dynamic column types.
399858 records were imported successfully into 'cov_data'!
MySQL connection has been closed.
